# Gen-UI-Kit Example with LangChain and AWS Bedrock

In this notebook, we will use the Gen-UI-Kit tool to generate a starter kit for an insurance underwriting app. We'll utilize LangChain's AWS Bedrock integration for LLM inference and parse the structured response using the JsonOutputToolsParser.

In [ ]:
#%pip install gen_ui_kit langchain_aws

In [ ]:
from langchain_aws import ChatBedrock
from gen_ui_kit import GenUIKit, GenUIKitTools
from langchain.output_parsers import JsonOutputToolsParser

### Set the inference modifiers and initialize the Bedrock client

We'll define the inference modifiers such as `max_tokens`, `temperature`, and other settings to control the behavior of the model.

In [ ]:
inference_modifier = {
    'max_tokens': 4096, 
    'temperature': 0.5,
    'top_k': 250,
    'top_p': 1,
    'stop_sequences': ['\n\nHuman']
}

llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", 
    model_kwargs=inference_modifier
)

### Set up the output parser and bind the Gen-UI-Kit tool to the LLM

We'll use `JsonOutputToolsParser` to parse the tool's structured output and bind the Gen-UI-Kit tool to the LLM.

In [ ]:
parser = JsonOutputToolsParser()

agent_gen_ui_tool_kit = llm.bind_tools([GenUIKitTools.GenUiKitLangchainTool])
chain = agent_gen_ui_tool_kit | parser

### Generate the starter kit for the insurance underwriting app idea

We'll provide a detailed description of the app and invoke the tool chain to generate the structured response.

In [ ]:
structured_response = agent_gen_ui_tool_kit.invoke('''create starter kit for the idea insurance underwriting app. Detailed description about this app followed as below:
    An Insurance Underwriting Web App is a digital platform designed to streamline and automate the process of underwriting insurance policies. Underwriting is the process by which insurers assess the risk of insuring a client and decide the terms and pricing of the insurance policy based on the client's risk profile. This web app would serve both insurance companies and underwriters by providing tools to evaluate risks, generate quotes, and manage applications more efficiently.''')

input_to_tool = {}
if structured_response.content == '':
    if structured_response.tool_calls:
        structured_response = structured_response.tool_calls[0]['args']
        if 'properties' in structured_response:
            input_to_tool = structured_response['properties']
        else:
            input_to_tool = structured_response
    else:
        print('invalid tool call')
else:
    print(structured_response.content)

### Run the tool using the parsed input

We'll initialize the GenUIKit tool using the parsed input and run it with the LLM to generate the UI components.

In [ ]:
agent_gen_ui_tool_instance = GenUIKit.GenUiKitLangchainTool(**input_to_tool)

agent_gen_ui_tool_instance._run(llm=llm)

Once the tool is executed, it will generate a UI kit that can be customized or directly connected to backends or APIs.